# Comparação de Modelos Preditivos

&emsp;A comparação de modelos busca identificar aquele com melhor desempenho para um problema específico, utilizando métricas que quantificam sua eficácia preditiva. Alguns modelos se sobressaem em determinadas métricas, enquanto outros se destacam em diferentes aspectos. Essa análise permite compreender os _trade-offs_ entre as métricas e selecionar o modelo que oferece o melhor equilíbrio entre precisão, viabilidade e facilidade de interpretação, facilitando a escolha mais adequada para o problema em questão.

## Comparação de Modelos Não Supervisionados

Os modelos preditivos não supervisionados identificam padrões em dados não rotulados, ao contrário dos supervisionados, que dependem de exemplos classificados. A comparação entre abordagens, como _K-means_ e _DBScan_, é essencial, pois cada algoritmo reage de maneira distinta à variabilidade, densidade dos dados e ruídos. Essa análise ajuda a escolher o modelo mais adequado, considerando a sensibilidade a outliers e a capacidade de lidar com dados complexos. Além disso, a avaliação contínua dos modelos permite otimizações, aprimorando a identificação de padrões e tendências em diferentes cenários.



**Ressalva importante:** esse notebook requer muito processamento, por isso, algumas células podem estar desativadas e pode ocasionar em erros de execução. Por favor, verifique se o ambiente de execução está configurado corretamente e se o hardware é suficiente para rodar o código. 

- Instalação das bibliotecas necessárias

Para mais informações de como rodar, veja o arquivo `orientacao.md`.

In [ ]:
%pip install pandas numpy matplotlib seaborn plotly scikit-learn

- _Importings_ das bibliotecas

In [20]:
import pandas as pd # pandas
import numpy as np # numpy
import matplotlib.pyplot as plt  # matplotlib
import seaborn as sns  # seaborn
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

- Leitura do _DataFrame_

In [21]:
df = pd.read_csv('./data_updated.csv')
features = ["faixa-etaria_encoded_standardized","valor-pago-sinistro_standardized", "descricao_servico_sinistro_encoded_standardized", "doenca_relacionadas_encoded_standardized", "sexo_encoded", "tipo_servico_encoded_standardized"]
X = df[features]

## Métricas de performance

&emsp;Como se trata de um modelo não supervisionado, métricas como acurácia, precisão, _recall_ e _F1-score_ não são aplicáveis para sua avaliação. Em vez disso, utilizamos três métricas: _Silhouette Score_, _Calinski-Harabasz Score_ e _Davies-Bouldin Score_. O _Silhouette Score_ mede a similaridade de um objeto com seu próprio cluster em comparação com outros clusters, variando de -1 a 1, sendo que valores próximos de 1 indicam melhor desempenho. O _Calinski-Harabasz Score_ calcula a razão entre a dispersão intra-cluster e a dispersão inter-cluster, com valores maiores indicando melhores resultados. Por fim, o _Davies-Bouldin Score_ avalia a média das similaridades entre cada _cluster_ e seu _cluster_ mais similar, onde valores menores são mais favoráveis.

## Clusterização: K-Means

&emsp;O primeiro modelo analisado foi o *_K-Means_*, um dos algoritmos de clusterização mais difundidos. O algoritmo se baseia na idéia de que cada _cluster_ possui um centroide, que representa o ponto médio de um grupo de dados. O algoritmo funciona iterativamente, ajustando os pontos centrais conforme novos dados são avaliados. Cada dado é alocado ao _cluster_ cujo centroide está mais próximo, utilizando uma medida de distância, como a euclidiana.

- Informações iniciais do _Dataframe_:

In [ ]:
df.info()

&emsp;O _Elbow Method_ é uma forma de encontrar a quantidade ideal de clusters (K ideal) para se utilizar no algoritimo K-Means, uma vez que o algoritimo precisa que a quantidade de _clusters_ que serão designados seja definido previamente.

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df[features])
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.grid()
plt.show()

&emsp; O código a sequir realiza a criação do modelo _K-Means_ para agrupar os dados numéricos em 4 clusters, sendo que a quantidade de _clusters_ foi definida anteriormente a partir do _Elbow Method_, uma técnica que auxilia na escolha do valor ideal de K. Após o treinamento do modelo, cada ponto é classificado em um dos 4 _clusters_, e os centros de cada _cluster_ são exibidos. Além disso, a quantidade de elementos pertencentes a cada _cluster_ é calculada e apresentada no final.

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0, max_iter=100000)
numeric_cols = df[features]

kmeans.fit(X)

df['cluster'] = kmeans.predict(X)
print(kmeans.cluster_centers_)
df["cluster"].value_counts()

&emsp;Para reduzir a dimensionalidade dos dados, foi utilizado o algoritmo PCA, que é uma técnica de redução dimensional projetada para simplificar grandes conjuntos de dados enquanto preserva o máximo de variabilidade possível. O PCA transforma os dados originais em novas variáveis, denominadas componentes principais (```PCA1``` e ```PCA2```), que são combinações lineares das variáveis originais.

&emsp;Dessa forma, os dados são reduzidos a 2 componentes principais para facilitar a visualização dos clusters formados pelo _K-Means_. Esses componentes são incorporados ao DataFrame e utilizados para criar um gráfico de dispersão, onde os pontos são coloridos de acordo com o _cluster_ a que pertencem. Os centróides dos _clusters_ também são plotados no gráfico, mostrando os centros dos grupos, como pode ser observado na visualização.


In [ ]:
# Aplicar PCA para reduzir os dados a 2 componentes principais
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Adicionar os componentes principais ao DataFrame para visualização
df['PCA1'] = X_pca[:, 0]
df['PCA2'] = X_pca[:, 1]

# Plotar os resultados em um gráfico de dispersão
plt.figure(figsize=(20, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='cluster', data=df, palette='viridis', style='cluster', markers=['o', 's', '^'])

# Plotar os centróides dos clusters em cima do gráfico PCA
centroids_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=300, c='red', marker='X', label='Centroides')

plt.title('Visualização dos Clusters com PCA e KMeans')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()

&emsp; Este _script_ verifica se algumas colunas estão presentes no DataFrame ```df```. Se estiverem, ele gera um gráfico de dispersão (_scatter plot_) onde os dados são coloridos de acordo com os _clusters_ definidos pelo K-Means. O gráfico exibe a relação entre a quantidade e o valor pago do sinistro, facilitando a visualização dos agrupamentos nos dados. Caso as colunas não estejam presentes, é exibida uma mensagem informando que as colunas necessárias estão ausentes.

In [ ]:

# Verificar se as colunas existem no DataFrame
if 'quantidade_standardized' in df.columns and 'valor-pago-sinistro_standardized' in df.columns and 'cluster' in df.columns:
    plt.figure(figsize=(10, 6))

    # Scatter plot com coloração pelos clusters
    sns.scatterplot(
        x="quantidade_standardized",
        y="valor-pago-sinistro_standardized",
        hue='cluster',
        data=df,
        palette='viridis',
        s=100
    )

    plt.title('Clusters de KMeans')
    plt.xlabel('Quantidade')
    plt.ylabel('Valor Pago Sinistro')
    plt.legend()
    plt.show()
else:
    print("As colunas necessárias não estão presentes no DataFrame.")

&emsp;Para uma visualização mais clara, foi criado um gráfico 3D que mostra os _clusters_ formados pelo _K-Means_. A visualização 3D dos _clusters_ é gerada utilizando o _Plotly Express_, começando com a criação de um _DataFrame_ chamado ```df_clusters```, que contém as colunas padronizadas e a coluna de clusters. Em seguida, é gerado um gráfico de dispersão com três dimensões, onde as coordenadas são baseadas em três variáveis padronizadas: ```faixa-etaria_encoded_standardized```, ```valor-pago-sinistro_standardized``` e ```descricao_servico_sinistro_encoded_standardized```.

In [ ]:
df_clusters = pd.DataFrame({
    'faixa-etaria_encoded_standardized': X['faixa-etaria_encoded_standardized'],
    'valor-pago-sinistro_standardized': X['valor-pago-sinistro_standardized'],
    'descricao_servico_sinistro_encoded_standardized': X['descricao_servico_sinistro_encoded_standardized'],
    'cluster': df['cluster']
})

fig = px.scatter_3d(
    df_clusters,
    x='faixa-etaria_encoded_standardized',
    y='valor-pago-sinistro_standardized',
    z='descricao_servico_sinistro_encoded_standardized',
    color='cluster',
    title='Clusters de KMeans em 3D',
    opacity=1.0,
    size_max=10
)

fig.show()

&emsp; Este código ajusta o _StandardScaler_ aos dados originais e, em seguida, utiliza-o para reverter a normalização dos centróides dos _clusters_, retornando-os ao espaço original. Os centróides são então convertidos em um DataFrame para facilitar a interpretação. O código exibe os centróides de cada _cluster_ no espaço original e mostra a contagem de elementos em cada _cluster_.

In [ ]:
# Fit the StandardScaler before using it to transform the data
scaler = StandardScaler()
scaler.fit(X)

# Obtain the centroids in the original space (before normalization)
centroids_original_space = scaler.inverse_transform(kmeans.cluster_centers_)

# Convert the centroids to a DataFrame for easier interpretation
centroids_df = pd.DataFrame(centroids_original_space, columns=numeric_cols.columns)

# Display the centroids of each cluster
print("Centroids of Clusters in the Original Space:")
print(centroids_df)

# View the count of elements in each cluster
print(df["cluster"].value_counts())


&emsp; Criação de uma matriz de dispersão (_scatter matrix_) usando _Plotly Express_ para visualizar a relação entre várias variáveis dos _clusters_. Cada ponto é colorido de acordo com o _cluster_ a que pertence, facilitando a comparação das variáveis dentro de cada _cluster_.

In [ ]:
fig_matrix = px.scatter_matrix(
    df_clusters,
    dimensions=['descricao_servico_sinistro_encoded_standardized', 'faixa-etaria_encoded_standardized', 'valor-pago-sinistro_standardized'],
    color='cluster',
    title='Scatter Matrix entre Variáveis por Cluster',
    labels={   # Definir rótulos claros para as variáveis
        'descricao_servico_sinistro_encoded_standardized': 'Código Serviço',
        'faixa-etaria_encoded_standardized': 'Faixa Etária',
        'valor-pago-sinistro_standardized': 'Valor Pago'
    },
    height=900,  # Aumentar a altura para reduzir sobreposição
    width=900    # Aumentar a largura
)

# Ajustando o layout para evitar sobreposição das labels
fig_matrix.update_layout(
    title={'x':0.5},  # Centraliza o título
    font=dict(size=10),  # Reduz o tamanho da fonte
    margin=dict(l=50, r=50, b=100, t=100),  # Ajusta as margens
    autosize=False
)

# Mostrar o gráfico
fig_matrix.show()


- Cálculo das métricas de performance

In [ ]:
# Aplicar o método de Silhouette, Davis-Bouldin e Calinski-Harabasz
silhouette = silhouette_score(X, df['cluster'])
print(f"Silhouette Score: {silhouette}")
davies_bouldin = davies_bouldin_score(X, df['cluster'])
print(f"Davies-Bouldin Score: {davies_bouldin}")
calinski_harabasz = calinski_harabasz_score(X, df['cluster'])
print(f"Calinski-Harabasz Score: {calinski_harabasz}")

- Cálculo da melhor combinação de hiperparâmetros para otimização do modelo usando _Grid Search_.

In [ ]:
def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return silhouette_score(X, labels)

# Função para calcular Calinski-Harabasz Score
def calinski_harabasz_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return calinski_harabasz_score(X, labels)

# Função para calcular Davies-Bouldin Score (menor é melhor)
def davies_bouldin_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return davies_bouldin_score(X, labels)


# Parâmetros para o GridSearchCV
param_grid = {
    'n_clusters': [4, 5, 6, 7, 8, 9, 10],  # Ampliar a variação de clusters
    'init': ['k-means++', 'random'],      # Testar inicializações diferentes
    'max_iter': [300, 500, 1000],         # Variar número de iterações
    'n_init': [10, 20, 30, 40],           # Testar diferentes inicializações
    'random_state': [0]                   # Manter a aleatoriedade fixa
}


# Instancia o modelo KMeans
kmeans = KMeans()

# Configurar o GridSearchCV para o Silhouette Score
grid_search_silhouette = GridSearchCV(estimator=kmeans, param_grid=param_grid,
                                      scoring=silhouette_scorer, cv=5, n_jobs=-1)

# Ajustar o modelo com os dados
grid_search_silhouette.fit(df[features])

# Melhor modelo e pontuação para Silhouette Score
print("Melhores Parâmetros (Silhouette):", grid_search_silhouette.best_params_)
print("Melhor Silhouette Score:", grid_search_silhouette.best_score_)

# Configurar o GridSearchCV para o Calinski-Harabasz Score
grid_search_calinski = GridSearchCV(estimator=kmeans, param_grid=param_grid,
                                    scoring=calinski_harabasz_scorer, cv=5, n_jobs=-1)

# Ajustar o modelo com os dados
grid_search_calinski.fit(df[features])

# Melhor modelo e pontuação para Calinski-Harabasz Score
print("Melhores Parâmetros (Calinski-Harabasz):", grid_search_calinski.best_params_)
print("Melhor Calinski-Harabasz Score:", grid_search_calinski.best_score_)

# Configurar o GridSearchCV para o Davies-Bouldin Score
grid_search_davies = GridSearchCV(estimator=kmeans, param_grid=param_grid,
                                  scoring=davies_bouldin_scorer, cv=5, n_jobs=-1)

# Ajustar o modelo com os dados
grid_search_davies.fit(df[features])

# Melhor modelo e pontuação para Davies-Bouldin Score
print("Melhores Parâmetros (Davies-Bouldin):", grid_search_davies.best_params_)
print("Melhor Davies-Bouldin Score:", -grid_search_davies.best_score_)

## Agglomerative Clustering

&emsp;O _Agglomerative Clustering_ é um algoritmo de clusterização hierárquica que agrupa os dados em _clusters_ de forma iterativa. Ele começa com cada ponto de dados como um _cluster_ separado e, em seguida, combina os _clusters_ mais próximos até que todos os pontos de dados estejam em um único _cluster_. O algoritmo é baseado em uma matriz de distâncias entre os pontos de dados, que é atualizada a cada iteração. O _Agglomerative Clustering_ é um algoritmo eficaz para identificar agrupamentos em dados não rotulados e pode ser usado para explorar a estrutura subjacente dos dados.

&emsp; Devido a demanda de muito poder computacional gerada pelo algoritmo, foi criado uma simplificação fracionada dos dados (```dfSample```) e aplicada uma técnica de redução de dimensionalidade (PCA) para a execução do modelo.

In [ ]:
dfSample = df[features].sample(frac=0.5)
X = dfSample.values


pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Printa o tamanho dos dados originais
print(X.shape)

- Criação do modelo

In [ ]:
#Esse demora pra carregar, portanto não deve ser rodado. 

print("Agglomerative Clustering")
clustering = AgglomerativeClustering(n_clusters=4, linkage="ward").fit_predict(X_pca)
print("Agglomerative Clustering done")


# Adicionando os clusters ao dataframe
dfSample['cluster'] = clustering
dfSample.to_csv('./data_updated_clusters_agg_50.csv', index=False) # Salvar processamento

fig = px.scatter_3d(
    dfSample,
    x='faixa-etaria_encoded_standardized',
    y='valor-pago-sinistro_standardized',
    z='quantidade_standardized',
    color='cluster',  # Colorir com base nos clusters
    title='Clusters de Agglomerative Clustering em 3D',
    opacity=1.0,
    size_max=10
)
fig.show()

- Visualização gráfica em 3d do modelo:

In [ ]:
df = pd.read_csv('./data_updated_clusters_agg_50.csv')
# print df shape
print(df.shape)


fig = px.scatter_3d(
    df,
    x='faixa-etaria_encoded_standardized',
    y='valor-pago-sinistro_standardized',
    z='quantidade_standardized',
    color='cluster',  # Colorir com base nos clusters
    title='Clusters de Agglomerative Clustering em 3D',
    opacity=1.0,
    size_max=10
)
fig.show()


- Cálculo das métricas de performance

In [ ]:
# Aplicar o método de Silhouette, Davis-Bouldin e Calinski-Harabasz
silhouette = silhouette_score(X, df['cluster'])
print(f"Silhouette Score: {silhouette}")
davies_bouldin = davies_bouldin_score(X, df['cluster'])
print(f"Davies-Bouldin Score: {davies_bouldin}")
calinski_harabasz = calinski_harabasz_score(X, df['cluster'])
print(f"Calinski-Harabasz Score: {calinski_harabasz}")

- Cálculo da melhor combinação de hiperparâmetros para otimização do modelo usando _Random Search_.

In [ ]:
# Definir função customizada para o silhouette score
def custom_silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)  # Realizar a predição
    score = silhouette_score(X, labels, metric='euclidean')  # Calcular o silhouette score
    return score

# Parâmetros para busca
param_distributions = {
    'n_clusters': [4, 5, 6, 7, 8, 9, 10],  # Número de clusters
    'linkage': ['complete', 'average', 'single'],
    'metric': ['euclidean', 'manhattan', 'cosine']  # O parâmetro correto é "metric"
}

# Inicializar o RandomizedSearchCV com AgglomerativeClustering
agg_cluster = AgglomerativeClustering()

random_search = RandomizedSearchCV(
    estimator=agg_cluster,
    param_distributions=param_distributions,
    n_iter=3,  # Número de combinações aleatórias a tentar
    scoring=custom_silhouette_scorer,  # Função customizada para o silhouette score
    cv=3,  # Validação cruzada (não muito relevante para clustering)
    verbose=1,
)

# Fazer o ajuste com os dados (X_pca é o conjunto de dados PCA transformado)
random_search.fit(X_pca)

# Resultados
print("Melhores parâmetros: ", random_search.best_params_)
print("Melhor score (silhouette): ", random_search.best_score_)
print("Davies-Bouldin Score: ", davies_bouldin_score(X, random_search.best_estimator_.labels_))
print("Calinski-Harabasz Score: ", calinski_harabasz_score(X, random_search.best_estimator_.labels_))
print("Melhor modelo: ", random_search.best_estimator_)


## DBSCAN

&emsp;O *_DBSCAN_* (_Density-Based Spatial Clustering of Applications with Noise_) é um algoritmo de agrupamento que se baseia na densidade dos pontos em um espaço, ao contrário de métodos como o *_K-Means_*, que utilizam distâncias para formar _clusters_. Ele é eficaz na detecção de clusters de forma arbitrária e na identificação de ruídos nos dados, classificando pontos isolados como _outliers_. Essa abordagem permite que o _*DBSCAN_* crie grupos compactos e bem definidos, sendo ideal para conjuntos de dados com formas complexas e distribuições desiguais.

- Funções utilizadas para a criação do modelo DBSCAN.

In [14]:
def dbscan(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    return db

def metricas_de_modelo(X, labels):
    silhouette = metrics.silhouette_score(X, labels)
    print("Silhouette Coefficient: %0.3f" %silhouette)
    
    calisnki = metrics.calinski_harabasz_score(X, labels)
    print("Calinski Harabasz Score: %0.3f" %calisnki)
    
    davies = metrics.davies_bouldin_score(X, labels)
    print("Davies Bouldin Score: %0.3f" %davies)

def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return silhouette_score(X, labels)

# Função para calcular Calinski-Harabasz Score
def calinski_harabasz_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return calinski_harabasz_score(X, labels)

# Função para calcular Davies-Bouldin Score (menor é melhor)
def davies_bouldin_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return davies_bouldin_score(X, labels)


- Cálculo da melhor combinação de hiperparâmetros para otimização do modelo usando _Grid Search_.

In [18]:
param_grid = {
    'eps': [0.001, 0.01, 0.1, 1, 5, 10],
    'min_samples': [1, 2, 3, 5, 10, 20]
}

dbscan = DBSCAN()

# Configurar o GridSearchCV para o Silhouette Score
grid_search_silhouette = GridSearchCV(estimator=dbscan, param_grid=param_grid,
                                      scoring=silhouette_scorer, cv=5, n_jobs=-1)

# Ajustar o modelo com os dados
grid_search_silhouette.fit(X)

# Melhor modelo e pontuação para Silhouette Score
print("Melhores Parâmetros (Silhouette):", grid_search_silhouette.best_params_)
print("Melhor Silhouette Score:", grid_search_silhouette.best_score_)

# Configurar o GridSearchCV para o Calinski-Harabasz Score
grid_search_calinski = GridSearchCV(estimator=dbscan, param_grid=param_grid,
                                    scoring=calinski_harabasz_scorer, cv=5, n_jobs=-1)

# Ajustar o modelo com os dados
grid_search_calinski.fit(X)

# Melhor modelo e pontuação para Calinski-Harabasz Score
print("Melhores Parâmetros (Calinski-Harabasz):", grid_search_calinski.best_params_)
print("Melhor Calinski-Harabasz Score:", grid_search_calinski.best_score_)

# Configurar o GridSearchCV para o Davies-Bouldin Score
grid_search_davies = GridSearchCV(estimator=dbscan, param_grid=param_grid,
                                  scoring=davies_bouldin_scorer, cv=5, n_jobs=-1)

# Ajustar o modelo com os dados
grid_search_davies.fit(X)

# Melhor modelo e pontuação para Davies-Bouldin Score
print("Melhores Parâmetros (Davies-Bouldin):", grid_search_davies.best_params_)
print("Melhor Davies-Bouldin Score:", grid_search_davies.best_score_)

- Cálculo das métricas de performance.

In [ ]:
db = dbscan(X, eps=1, min_samples=10)
labels_dbscan = db.labels_
metricas_de_modelo(X, labels_dbscan)

- Análise das métricas do DBSCAN sem _outliers_ (ruído)

In [ ]:
mask = labels_dbscan != -1  
X_no_noise = X[mask]
labels_no_noise = labels_dbscan[mask]

metricas_de_modelo(X_no_noise, labels_no_noise)

- Plotagem do gráfico usando PCA.

In [ ]:
# aplicado o PCA para reduzir a dimensionalidade e possibilidade a execução do modelo.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(16, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels_dbscan, cmap='viridis', s=50)
plt.title('Spectral Clustering com PCA - 2 clusters')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

## Gaussian Mixture Model

&emsp;O _Gaussian Mixture Model_ (GMM) é um modelo probabilístico utilizado para clusterização, que parte do princípio de que os dados foram gerados a partir de uma combinação de várias distribuições Gaussianas. Cada distribuição Gaussiana representa um _cluster_ nos dados, e o GMM busca identificar esses clusters ajustando as distribuições em relação aos dados observados. Ao contrário de algoritmos de clusterização como o *_K-Means_*, que designam cada ponto de dados a um cluster específico, o GMM oferece maior flexibilidade ao atribuir uma probabilidade de pertencimento a cada cluster. Assim, um ponto de dados pode estar associado a múltiplos _clusters_, com diferentes níveis de probabilidade.

- Funções utilizadas para a criação do modelo DBSCAN.

In [5]:
# Definindo a função para o Gaussian Mixture Model
def gaussian_mixture_model(X, n_components):
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    labels = gmm.fit_predict(X)
    return labels

### BIC - Bayesian Information Criterion

&ensp;A _Bayesian Information Criterion_ (BIC) é uma métrica usada para seleção de modelos preditivos e aprendizado de máquina. A BIC ajuda a comparar diferentes modelos com diferentes graus de complexidade, penalizando aqueles que possuem muitos parâmetros, para evitar _overfitting_ (ajuste excessivo aos dados). O melhor modelo, seria aquele que teoricamente possísse um valor menor de BIC, mas ao plotar em um gráfico os valores com seus respectivos parâmetros, podemos visualizar qual o que atinge um bom valor de BIC, mas com o menor valor de parâmetro ```n_components```, sendo um ponto bem onde os _clusters_ tem uma boa qualidade, mas com menor complexidade.

In [6]:
def bic(X):
    n_components_range = range(1, 14)
    bic_scores = []

    for n in n_components_range:
        gmm = GaussianMixture(n_components=n, random_state=42)
        gmm.fit(X)
        
        bic_scores.append(gmm.bic(X))

    # Plot AIC and BIC to find the optimal number of components
    plt.figure(figsize=(12, 6))
    plt.plot(n_components_range, bic_scores, label='BIC')
    plt.xlabel('Número de Componentes')
    plt.ylabel('BIC')
    plt.legend()
    plt.show()

- Aplicando o método BIC na variável ```X```

In [ ]:
bic(X)

- Aplicando o valor de ```n_components```

In [ ]:
labels_gmm = gaussian_mixture_model(X, n_components=2)
metricas_de_modelo(X, labels_gmm)

- Aplicação do PCA para redução da dimensionalidade.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(16, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels_gmm, cmap='viridis', s=50)
plt.title('Spectral Clustering com PCA - 2 clusters')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

## Conclusão

&ensp;Para escolher o melhor modelo de clustering, analisamos três métricas principais: Calinski-Harabasz Index, Silhouette Score e Davies-Bouldin Index.

&ensp;O Calinski-Harabasz Index, que mede a qualidade dos clusters com base nas distâncias intra e inter-cluster, não foi utilizado para comparar modelos, pois é sensível ao número de clusters e pode ser influenciado por outliers, não sendo uma boa métrica de comparação entre diferentes modelos.

&ensp;O Silhouette Score, que mede a coesão dos clusters, mostrou que o DBSCAN (0.830) e o Agglomerative Clustering (0.721) tiveram o melhor desempenho, formando clusters bem definidos. Já KMeans(0.580) e GMM tiveram valores menores, indicando menos coesão.

&ensp;O Davies-Bouldin Index, que avalia a separação entre clusters, mostrou que o DBSCAN teve o melhor desempenho (0.191), com clusters bem separados e baixa sobreposição. O Agglomerative Clustering também se saiu bem (0.455), enquanto GMM (3.045) e KMeans (0.202) apresentaram maior sobreposição.

&ensp;Portanto, o DBSCAN foi o modelo com melhor desempenho geral, no entanto, analisando a coerência dos clusters, o KMeans foi o que mais se destacou e portanto é o modelo mais indicado para o problema em questão.